In [1]:
'''
This notebook is used to update the transf() subroutine.
'''

'\nThis notebook is used to update the transf() subroutine.\n'

In [6]:
%matplotlib inline
import os
import itertools
import importlib

import pandas as pd

In [17]:
def load_data():
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation/crd',
                         'LW/examples/separate_g_groups',
                         'study__lblnew_g1_threshold',
                         'compute_transf_table/output_transf.dat')
    df = pd.read_csv(fpath, sep=r'\s+')
    return df


In [55]:
def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4, dtype=float):
    '''
    '''
    if dtype == float:
        strfmt = '{:15.6e}_r8'
    elif dtype == int:
        strfmt = '{:15d}'
    else:
        raise ValueError('dtype must be either float or int.')
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines


def vector_to_F77(array=None, num_values_per_line=None, dtype=None):
    lines = vector_to_F77list(array=array, 
                              num_values_per_line=num_values_per_line,
                              dtype=dtype)
    
    rlines = [5 * ' ' + '&' + l for l in lines]
    
    fortran = '\n'.join(rlines)
    return fortran

In [68]:
vs = range(3500)
for i in range(0, 3500, 1500):
    print(vs[i: i + 1500])

range(0, 1500)
range(1500, 3000)
range(3000, 3500)


In [78]:
def ei3():
    
    df = load_data()
    
    nv_sec = 1500
    
    ls = []
    for i in range(0, len(df), nv_sec):
        vs = df['transf'][i: i + nv_sec]
        s = vector_to_F77(vs, num_values_per_line=3, dtype=float)
        
        istart = i + 1
        ifinal = len(df) if i + nv_sec > len(df) else i + nv_sec
        s_indexrange = '{}:{}'.format(istart, ifinal)
        
        s = '\n'.join(['ei3({}) = (/'.format(s_indexrange),
                       s,
                       5 * ' ' + '&' + '/)'])        
        
        ls.append(s)
        
    return ls


def subroutine():
    ls = ('subroutine transf (dx,tku)',
          'implicit none',
          'integer, parameter :: r8 = kind(1.0d0)',
          '',
          '! input the pre-computed data with dtau = 0.002',
          '!   and tau ranging from 0 to 7.',
          '',
          'integer, parameter :: ntau = 3500',
          'real*8 ei3(ntau)',
          'real*8 dx, tku, fz',
          'integer iz')
          
    ls1 = ('if (dx .lt. 7.0_r8) then',                                                    
           '  iz =dx/ 0.002_r8 +1.0_r8',                                                   
           '  fz = dx-float(iz-1) * 0.002_r8',                                            
           '  tku = ei3(iz) + fz * (ei3(iz+1) - ei3(iz)) /0.002_r8',
           'else',                                                                        
           '  tku = 0._r8',                                                           
           'endif',
           '',
           'return',
           'end')
    
    lines = list(ls)
    lines = lines + ['']
    lines = lines + ei3()
    lines = lines + ['']
    lines = lines + list(ls1)
    return lines


def file_content():
    lines = subroutine()
    lines = [6 * ' ' + l for l in lines]
    s = '\n'.join(lines)
    return s


def script():
    s = file_content()
    fpath = os.path.join('trans.f')
    with open(fpath, mode='w', encoding='utf-8') as f:
        f.write(s)

In [79]:
print(script())

None


In [77]:
df['transf'][0: 5]

0    1.000000
1    0.996028
2    0.992103
3    0.988217
4    0.984368
Name: transf, dtype: float64